# Import Required Libraries
Import the necessary libraries, including torch, torchvision, and matplotlib.

In [3]:
# Import Required Libraries

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10 Dataset
Load the CIFAR-10 dataset using torchvision.datasets and apply necessary transformations.

In [5]:
# Load CIFAR-10 Dataset

# Define transformations for the training and test sets
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the training set
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Load the test set
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define the classes in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Function to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Load CIFAR-10 Dataset

# Define transformations for the training and test sets
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the training set
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Load the test set
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define the classes in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Function to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

# Introduce Corruption to Dataset
Introduce corruption to the dataset by applying random noise or other distortions to the images.

In [ ]:
# Introduce Corruption to Dataset

import random

def add_noise(img, noise_factor=0.2):
    """Add random noise to an image."""
    noise = torch.randn(img.size()) * noise_factor
    noisy_img = img + noise
    noisy_img = torch.clip(noisy_img, 0., 1.)
    return noisy_img

# Apply noise to the training set
corrupted_trainset = []
for img, label in trainset:
    noisy_img = add_noise(img)
    corrupted_trainset.append((noisy_img, label))

# Convert the list to a DataLoader
corrupted_trainloader = torch.utils.data.DataLoader(corrupted_trainset, batch_size=4,
                                                    shuffle=True, num_workers=2)

# Get some random corrupted training images
dataiter = iter(corrupted_trainloader)
corrupted_images, corrupted_labels = dataiter.next()

# Show corrupted images
imshow(torchvision.utils.make_grid(corrupted_images))
# Print labels
print(' '.join(f'{classes[corrupted_labels[j]]:5s}' for j in range(4)))

# Preprocess the Dataset
Preprocess the dataset by normalizing the images and converting them to tensors.

In [ ]:
# Preprocess the Dataset

# Normalize the images and convert them to tensors
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Apply the transformations to the corrupted training set
preprocessed_trainset = [(transform(img), label) for img, label in corrupted_trainset]

# Convert the list to a DataLoader
preprocessed_trainloader = torch.utils.data.DataLoader(preprocessed_trainset, batch_size=4,
                                                       shuffle=True, num_workers=2)

# Get some random preprocessed training images
dataiter = iter(preprocessed_trainloader)
preprocessed_images, preprocessed_labels = dataiter.next()

# Show preprocessed images
imshow(torchvision.utils.make_grid(preprocessed_images))
# Print labels
print(' '.join(f'{classes[preprocessed_labels[j]]:5s}' for j in range(4)))

# Dataset Analytics
Analyze the dataset by visualizing sample images, calculating dataset statistics, and displaying class distributions.

In [ ]:
# Dataset Analytics

# Visualize sample images from the preprocessed dataset
dataiter = iter(preprocessed_trainloader)
images, labels = dataiter.next()

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

# Calculate dataset statistics
def calculate_statistics(loader):
    mean = 0.0
    std = 0.0
    total_images_count = 0
    for images, _ in loader:
        batch_samples = images.size(0)  # batch size (the last batch can have smaller size)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count
    return mean, std

mean, std = calculate_statistics(preprocessed_trainloader)
print(f'Mean: {mean}')
print(f'Std: {std}')

# Display class distributions
def display_class_distribution(loader):
    class_count = [0] * len(classes)
    for _, labels in loader:
        for label in labels:
            class_count[label] += 1

    plt.bar(classes, class_count)
    plt.xlabel('Classes')
    plt.ylabel('Number of images')
    plt.title('Class distribution in the preprocessed dataset')
    plt.show()

display_class_distribution(preprocessed_trainloader)